### Set the auth toke and the DA site and the SharePoint site

In [3]:
const token = 'My auth token';
const daSite = 'da-dc';
const spSite = 'dc';
const content = 'live';   // live or preview

async function logError(response) {
  console.error(`Request failed with status: ${response.status} ${response.statusText}`);
  
  console.error('Response Headers:');
  for (const [key, value] of response.headers.entries()) {
    console.error(`  ${key}: ${value}`);
  }
  
  try {
    const errorData = await response.json();
    console.error('Error Response Body:', errorData);
  } catch (e) {
    const errorText = await response.text();
    console.error('Error Response Body (text):', errorText);
  }
  
  throw new Error(`HTTP ${response.status}: ${response.statusText}`);
}

### Check the token

In [ ]:
const responseSP = await fetch(`https://admin.hlx.page/status/adobecom/${spSite}/main/`, {
  headers: {
    'x-auth-token': token,
  }
});

if (!responseSP.ok) {
  logError(responseSP);
} else {
  console.log(`${spSite} is OK`);
}

const responseDA = await fetch(`https://admin.hlx.page/status/adobecom/${daSite}/main/`, {
  headers: {
    'x-auth-token': token,
  }
});

if (!responseDA.ok) {
  logError(responseDA);
} else {
  console.log(`${daSite} is OK`);
}

### Get the file lists from source and target sites

In [ ]:
import { scheduler } from "node:timers/promises";

const sites = [daSite, spSite];
const files = {};

for (const site of sites) {
  const response = await fetch(`https://admin.hlx.page/status/adobecom/${site}/main/*`, {
    method: 'POST',
    headers: {
      'x-auth-token': token,
      'Content-Type': 'application/json'
    },

    body: JSON.stringify({
      "select": [
        content
      ],
      "paths": [
        "/*"
      ],
      "pathsOnly": true
    })
  });

  if (!response.ok) {
    logError(response);
  }

  const data = await response.json();

  const jobLink = data.links.self;
  let jobDetails;

  let retry = 10;
  while (retry > 0) {
    const responseState = await fetch(jobLink, {
      headers: {
        'x-auth-token': token
      }
    });

    if (!responseState.ok) {
      logError(responseState);
    }

    const dataState = await responseState.json();
    //console.log(dataState);

    if (dataState.state === 'stopped') {
      jobDetails = dataState.links.details;
      break;
    }

    await scheduler.wait(5000)
    retry -= 1;
  }

  const responseDetail = await fetch(jobDetails, {
    headers: {
      'x-auth-token': token
    }
  });

  if (!responseDetail.ok) {
    logError(responseDetail);
  }

  const dataDetail = await responseDetail.json();
  const fileList = dataDetail.data.resources[content];
  files[site] = fileList;

  console.log(`${content} page and asset count: ${fileList.length}`);

  const fileName = `${site}_${content}.txt`;
  await Deno.writeTextFile(fileName, fileList.join('\n'));
  console.log(`\nOutput saved to: ${fileName}`);
}

### Find the differences

In [ ]:
const spSet = new Set(files[spSite]);
const daSet = new Set(files[daSite]);

const spOnly = [...spSet.difference(daSet)];
const daOnly = [...daSet.difference(spSet)];

console.log(`Count of SP only files: ${spOnly.length}`);
console.log(`Count of DA only files: ${daOnly.length}`);

await Deno.writeTextFile(`${spSite}_only.txt`, spOnly.join('\n'));
await Deno.writeTextFile(`${daSite}_only.txt`, daOnly.join('\n'));


### Analyze SharePoint Only Files

In [ ]:
// Not interested in these files
const filters = ['query-index', 'sitemap.xml', 'sitemap-'];

const missingFiles = spOnly.filter(x =>
  !filters.some(filter => x.includes(filter)));

await Deno.writeTextFile(`${spSite}_only_filtered.txt`, missingFiles.join('\n'));

console.log(`Count of SP only files after filtered: ${missingFiles.length}`);

### Check Redirects

In [ ]:
const status301 = [];
const statusMap = {};
let count = 0;

for (const path of missingFiles) {
  count += 1;
  const url = `https://main--${spSite}--adobecom.aem.live${path}`;
  const resp = await fetch(url, {
    method: 'HEAD',
    redirect: 'manual', 
  })

  if (resp.status === 301) {
    status301.push(`${url},${resp.headers.get('location')}`);
  } else if (statusMap[resp.status]) {
    statusMap[resp.status].push(url);
  } else {
    statusMap[resp.status] = [url];
  }
  if (count % 100 === 0) {
    console.log(`${count}/${missingFiles.length}`);
  }
}

await Deno.writeTextFile(`${spSite}_missing_redirects.txt`, status301.join('\n'));
console.log(`${spSite}_missing_redirects.txt`);

for (const key of Object.keys(statusMap)) {
  await Deno.writeTextFile(`${spSite}_missing_${key}.txt`, statusMap[key].join('\n'));
  console.log(`${spSite}_missing_${key}.txt`);
}